In [85]:
import pandas as pd
from sklearn.utils import shuffle
import re ,nltk
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,cross_val_score

In [104]:
def Preprocessing(Tweets):
    processed=[]
    StopWords={sw for sw in nltk.corpus.stopwords.words('english')} #nltk.corpus.stopwords.words('english')
    s={'\'s','i\'d','he\'s'}
    StopWords=StopWords|s
    
    pattern=re.compile(r'https?[^ ]+|@\w+|[^a-zA-Z#\' ]') #remove username and hyperlinks
    for tweet in Tweets:
        if len(tweet)==0:continue
        tweet=re.sub(r'<.*?>','',tweet) #remove html tags
        tweet=pattern.sub('',tweet).lower().split()
        tweet=' '.join([w for w in tweet if not w in StopWords])
        Root=nltk.stem.PorterStemmer()
        RootWords=[Root.stem(w) for w in tweet.split()]
        tweet=str(' '.join(RootWords))
        tweet=tweet.replace("'","").replace("\"","").replace("#","")
        processed.append(tweet)
    return processed   

In [100]:
df_Obama_tweets=df_Obama_tweets=pd.read_excel('trainingObamaRomneytweets.xlsx',sheet_name='Obama')
df_Romney_tweets=pd.read_excel('trainingObamaRomneytweets.xlsx',sheet_name='Romney')

# retains first 4 columns
df_Obama_tweets=df_Obama_tweets.iloc[1:,0:4]
df_Romney_tweets=df_Romney_tweets.iloc[1:,0:4]

#rename the colums
df_Obama_tweets.columns=['Date','Time','Tweets','Class']
df_Romney_tweets.columns=['Date','Time','Tweets','Class']
df_Obama_tweets.index=range(1,len(df_Obama_tweets)+1)
df_Romney_tweets.index=range(1,len(df_Romney_tweets)+1)

#drop mixed and !!! class for now(we may consider it later)
df_Romney_tweets=df_Romney_tweets[(df_Romney_tweets['Class'].isin((0,1,-1)))]
df_Obama_tweets=df_Obama_tweets[(df_Obama_tweets['Class'].isin((0,1,-1)))]

#drop empty tweet rows
df_Obama_tweets.dropna(subset=['Tweets'],inplace=True)
df_Romney_tweets.dropna(subset=['Tweets'],inplace=True)

#shuffle the dataframe to avoid bias
df_Obama_tweets=shuffle(df_Obama_tweets).reset_index(drop=True)
df_Romney_tweets=shuffle(df_Romney_tweets).reset_index(drop=True)

#raw tweet and class
Obama_tweet=df_Obama_tweets['Tweets'].tolist()
Obama_class=df_Obama_tweets['Class'].tolist()

Romney_tweet=df_Romney_tweets['Tweets'].tolist()
Romney_class=df_Romney_tweets['Class'].tolist()

In [103]:
# preprocess the tweets
Xtrain_Obama=Preprocessing(Obama_tweet)
Xtrain_Romney=Preprocessing(Romney_tweet)

#Vectorize the tweet 
vect=TfidfVectorizer()
Xtrain_tfidf=vect.fit_transform(Xtrain_Obama)

#initialize the classifiers
clf=MultinomialNB()

#score on training data
cross_val_score(clf,Xtrain_tfidf,Obama_class,cv=10).mean()


0.5817135085131088